In [9]:
import numpy as np 
import pandas as pd 
from functools import reduce
import operator
from scipy import stats


In [10]:
def confidence95(metric_list):
    # Step 1: Calculate the mean
    mean = np.mean(metric_list)
    # Step 2: Calculate the standard error of the mean (SEM)
    sem = stats.sem(metric_list)
    # Step 3: Find the critical value for a 95% confidence interval
    confidence_level = 0.95
    degrees_of_freedom = len(metric_list) - 1
    critical_value = stats.t.ppf((1 + confidence_level) / 2, degrees_of_freedom)

    # Step 4: Calculate the margin of error
    margin_of_error = critical_value * sem
    return margin_of_error

In [11]:
LOG_DIR = "/home/chri6578/Documents/gttp/logs/evalcheck/"

datasets = [ "wikipedia", "reddit", "uci"]
# datasets = ["CanParl" "Flights" "mooc" "SocialEvo" "UNtrade" 
# "Contacts" "enron" "lastfm" "reddit" "uci" "UNvote" "wikipedia"]

metrics = ['acc', 'auc', 'ap']
splits = ['val', 'test']
models = ['TGAT', 'JODIE', 'DyRep', 'TGN', 'CAWN']

names_list ={
    'test': ['model', 'distort', 'sample', 'split', 'inference', 'acc', 'auc', 'ap'],
    'val': ['model', 'distort', 'sample', 'split', 'inference', 'acc', 'auc', 'ap', 'ep']
}


In [17]:
dataset = datasets[2]
split = splits[1]
model = models[4] 

In [18]:
file_name = LOG_DIR+dataset+f'_{split}.log'
log_file = pd.read_csv(file_name, delimiter='\t',
                    names= names_list[split])

log_file['distort'] = log_file['distort'].fillna('none')

In [28]:
RESULT = {}

for metric in metrics:
    metric_list = list(log_file[(log_file['model']==model)&(log_file['inference']=='tdv')&(log_file['distort']=='none')][metric][1:])
    RESULT[metric] = (np.mean(metric_list), confidence95(metric_list))
    
metric_all = [ [f"{RESULT[metric][0]:.4f}", f"{RESULT[metric][1]:.1e}"] for metric in metrics]
metric_all = reduce(operator.concat, metric_all)
    

In [29]:
model, dataset

('CAWN', 'uci')

In [30]:
metric_all

['0.7695', '6.5e-03', '0.8495', '7.0e-03', '0.8866', '2.4e-03']